In [17]:
!pip install opencv-python
!pip install opencv-contrib-python

!pip install ffmpeg moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=21761c5ae904be7f040fd118a2fe403bab61711d4c33203b68b8aa4fa1ddfbd5
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [18]:
import sys
import cv2
import numpy as np

import math
import moviepy.editor as mp
from operator import itemgetter

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3571712/45929032 bytes (7.8%)7503872/45929032 bytes (16.3%)11173888/45929032 bytes (24.3%)14942208/45929032 bytes (32.5%)18358272/45929032 bytes (40.0%)21233664/45929032 bytes (46.2%)24936448/45929032 bytes (54.3%)27885568/45929032 bytes (60.7%)31358976/45929032 bytes (68.3%)34611200/45929032 bytes (75.4%)38428672/45929032 bytes (83.7%)42205184/45929032 bytes (91.9%)

In [19]:
def find_speaking(audio_clip, window_size=20, volume_threshold=0.0001):

    num_windows = math.floor(audio_clip.end/window_size)

    window_is_silent = []
    for i in range(num_windows):
        s = audio_clip.subclip(i * window_size, (i + 1) * window_size)

        v = s.max_volume()
        window_is_silent.append(v < volume_threshold)

    start = 0
    end = 0
    speaking_intervals = []
    for i in range(1, len(window_is_silent)):
        if window_is_silent[i - 1] and not window_is_silent[i]:
            start = i * window_size

        if not window_is_silent[i - 1] and window_is_silent[i]:
            end = i * window_size
            new_speaking_interval = [start - 0.25, end + 0.25]

            need_to_merge = len(speaking_intervals) > 0 and speaking_intervals[-1][1] > new_speaking_interval[0]
            if need_to_merge:
                merged_interval = [speaking_intervals[-1][0], new_speaking_interval[1]]
                speaking_intervals[-1] = merged_interval
            else:
                element = (new_speaking_interval, new_speaking_interval[1] - new_speaking_interval[0])
                speaking_intervals.append(element)

    return speaking_intervals

In [20]:
def find_longest_speaking_interval(speaking_intervals):    
    maximum = max(speaking_intervals, key=itemgetter(1))[0] 
    return maximum 

In [21]:
clip = mp.VideoFileClip('drive/MyDrive/dataset/2022-04-11 14_31 Software Design, лекция, ВШЭ3, весна 2022.mp4')
audio = clip.audio

In [22]:
result = find_longest_speaking_interval(find_speaking(audio))

In [27]:
def find_shot_transition(cap):
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    threshold = 125

    last_mean = 0 
    start_time = cv2.getTickCount()
    shot_timecodes = []

    while True:
        (rv, im) = cap.read()
        if not rv:
            break

        frame_mean = np.sum(im) / float(im.shape[0] * im.shape[1] * im.shape[2])

        if cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0 >= result[0]:
            break

        if frame_mean >= threshold and last_mean < threshold:
            shot_timecodes.append(cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0)

        elif frame_mean < threshold and last_mean >= threshold:
            shot_timecodes.append(cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0)

        last_mean = frame_mean
    
    return shot_timecodes


In [28]:
clip = mp.VideoFileClip('drive/MyDrive/dataset/2022-04-11 14_31 Software Design, лекция, ВШЭ3, весна 2022.mp4')
audio = clip.audio

cap = cv2.VideoCapture('drive/MyDrive/dataset/2022-04-11 14_31 Software Design, лекция, ВШЭ3, весна 2022.mp4')

In [29]:
print(find_shot_transition(cap))
cap.release()

[31.8, 33.64, 33.88, 34.440000000000005, 35.64, 36.4, 580.32]
